### Here I will try running RRCF parallely and create the final dataset that the agent would see

In [3]:
import sys
import pandas as pd
import numpy as np
from pathlib import Path
from joblib import Parallel, delayed

In [6]:
current_dir = Path().cwd()
project_root = current_dir.parent
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))
    print('Done')

Done


In [8]:
from src.sync.RRCF_TS_anomaly_detection import get_data_ready, run_rrcf
from src.utils.save_data import load_df_parquet_safe

In [9]:
path = project_root / "data" / "processed" / "merge" / "final_smooth_rz_data.parquet"
df=load_df_parquet_safe(path=str(path))

In [10]:
df.head()

,Time,blink_intensity_smooth_rz,gaze_magnitude_smooth_rz,jaw_magnitude_smooth_rz,smile_intensity_smooth_rz,loudness_db_smooth_rz,pitch_relative_st_smooth_rz,pitch_expressiveness_st_smooth_rz,wps_smooth_rz,words,text_concat,speaker,filler_percentage,pause_percent_pr
0,0.0,3.571464,1.555960,-0.351862,1.418536,NaN,NaN,NaN,NaN,"[We're, starting]",We're starting,None,0.064433,0.079177
1,0.5,1.740830,1.323856,-0.425948,1.438164,NaN,NaN,NaN,NaN,"[now., [*]]",now. [*],A,0.064433,0.079177
2,1.0,1.297544,1.195807,-0.627946,1.056126,NaN,NaN,NaN,NaN,None,,A,0.064433,0.079177
3,1.5,1.004625,0.428483,-0.742188,0.602340,NaN,NaN,NaN,NaN,"[So, welcome]",So welcome,A,0.064433,0.079177
4,2.0,1.322258,1.213327,-0.828038,0.355089,NaN,NaN,NaN,NaN,"[to, the]",to the,A,0.064433,0.079177


In [22]:
feature_cols = [
    'blink_intensity_smooth_rz',
    'gaze_magnitude_smooth_rz',
    'smile_intensity_smooth_rz',
]

features = []
for feature in feature_cols:
    features.append(get_data_ready(data=df, features=[feature], type='ui'))

In [26]:
features[0][1]

array([[3.57146417],
       [1.74083022],
       [1.29754446],
       ...,
       [1.6549074 ],
       [1.252199  ],
       [1.04397284]], shape=(1264, 1))

In [27]:
results = Parallel(n_jobs=-1)(
    delayed(run_rrcf)(
        features=feature[1],
        shingle=10
    )
    for feature in features
)

anomaly_scores_by_feature = dict(zip(feature_cols, results))

In [ ]:
anomaly_scores_by_feature